### ATR 3X Strategy

In [1]:
import time
from datetime import datetime
import vectorbt as vbt

import pandas as pd
import pandas_ta as ta
import numpy as np
import mplfinance as mpf

### Import CSV

In [50]:
data = pd.read_csv('USDJPY_5m.csv')
data.tail()

,timestamp,open,high,low,close,adj close,volumn
35636,2022-04-29 17:35:00+01:00,129.779999,129.834000,129.759995,129.809998,129.809998,0
35637,2022-04-29 17:40:00+01:00,129.807007,129.811996,129.729996,129.766998,129.766998,0
35638,2022-04-29 17:45:00+01:00,129.772995,129.783997,129.710007,129.737000,129.737000,0
35639,2022-04-29 17:50:00+01:00,129.735001,129.768005,129.639999,129.669998,129.669998,0
35640,2022-04-29 17:55:00+01:00,129.679993,129.690994,129.600006,129.671997,129.671997,0


In [51]:
df = data.copy()

### Create strategy

In [6]:
atr_3x = ta.Strategy(
    name = 'Gloden cross',
    description = 'Gloden cross by EMA 2 Line',
    ta = [
        {'kind': 'atr', 'length': 14},
        {'kind': 'ema', 'length': 260}
    ]
)
df.ta.strategy(atr_3x)
df

,timestamp,open,high,low,close,adj close,volumn,ATRr_14,EMA_260
0,2021-11-04 17:00:00+00:00,1.349364,1.349364,1.348418,1.348963,1.348963,0,NaN,NaN
1,2021-11-04 17:15:00+00:00,1.349000,1.349400,1.348981,1.349273,1.349273,0,NaN,NaN
2,2021-11-04 17:30:00+00:00,1.349400,1.349437,1.348563,1.349109,1.349109,0,NaN,NaN
3,2021-11-04 17:45:00+00:00,1.348781,1.349273,1.348527,1.349273,1.349273,0,NaN,NaN
4,2021-11-04 18:00:00+00:00,1.349382,1.349546,1.349182,1.349546,1.349546,0,NaN,NaN
...,...,...,...,...,...,...,...,...,...
11920,2022-04-29 16:45:00+01:00,1.256329,1.256929,1.256187,1.256581,1.256581,0,0.001462,1.255084
11921,2022-04-29 17:00:00+01:00,1.256329,1.256644,1.255824,1.255950,1.255950,0,0.001416,1.255091
11922,2022-04-29 17:15:00+01:00,1.256281,1.257719,1.256281,1.256787,1.256787,0,0.001441,1.255104
11923,2022-04-29 17:30:00+01:00,1.256266,1.257150,1.256266,1.256913,1.256913,0,0.001401,1.255118


### Create signal

In [48]:
class BackTestATR3X:
    def __init__(self, params):
        self.orders = pd.DataFrame(columns = ['action', 'open time', 'open', 'close time', 'close', 'T/P', 'S/L', 'result'])
        self.order = {'action': None, 'open time': None, 'open': None, 'close time': None, \
                      'close': None, 'T/P': None, 'S/L': None,'result': None}
        self.unknow_result = []
        
        self.current_action = 'close'
        self.atr = params['atr']
        self.ema = params['ema']
        
        self.pip_value = params['pip_value']
        self.rr = params['rr']
        self.df = params['df']
        self.plot = params['plot']

    def reset_order(self):
        self.order = {'action': None, 'open time': None, 'open': None, 'close time': None, \
                      'close': None, 'T/P': None, 'S/L': None,'result': None}
        
    def get_df(self):
        return self.df
    
    def plot_order(self, order):
        pass
    
    def backtest(self):
        atr_3x = ta.Strategy(
            name = 'Gloden cross',
            description = 'Gloden cross by EMA 2 Line',
            ta = [
                {'kind': 'atr', 'length': self.atr},
                {'kind': 'ema', 'length': self.ema}
            ]
        )
        self.df.ta.strategy(atr_3x)
        self.df = self.df.copy().dropna()
        self.df['timestamp'] = pd.to_datetime(self.df['timestamp'])
        self.df['action'] = None
        
        self.df['entries'] = False
        self.df['exits'] = False
        self.df['short_entries'] = False
        self.df['short_exits'] = False
        self.df['price_action'] = self.df['close']
        
        for index, row in self.df.iterrows():
            entry = 0
            
            if self.current_action == 'close':
                if (row['EMA_'+str(self.ema)] < row['close']) and (row['ATRr_'+str(self.atr)]*3 < (row['high']-row['low']) ):
                    self.df.at[index, 'action'] = 'buy'
                    self.df.at[index, 'entries'] = True
                    self.df.at[index, 'price_action'] = row['open']
                    self.order['action'] = 'buy'
                                
                    self.order['open time'] = row['timestamp']
                    self.order['open'] = row['open']
                    self.order['S/L'] = row['open'] - row['ATRr_'+str(self.atr)]
                    self.order['T/P'] = row['open'] + (self.rr * abs(row['open'] - self.order['S/L']))
                    self.current_action = 'buy'
                    
                elif (row['EMA_'+str(self.ema)] > row['close']) and (row['ATRr_'+str(self.atr)]*3 < (row['high']-row['low']) ):
                    self.df.at[index, 'action'] = 'sell'
                    self.df.at[index, 'short_entries'] = True
                    self.df.at[index, 'price_action'] = row['open']
                    self.order['action'] = 'sell'
                                
                    self.order['open time'] = row['timestamp']
                    self.order['open'] = row['open']
                    self.order['S/L'] = row['open'] + row['ATRr_'+str(self.atr)]
                    self.order['T/P'] = row['open'] - (self.rr * abs(row['open'] - self.order['S/L']))
                    self.current_action = 'sell'
        
            if self.current_action == 'buy':
                if row['low'] <= self.order['S/L'] and row['high'] >= self.order['T/P']:
                    self.unknow_result.append(row)
                    
                if row['low'] <= self.order['S/L']:
                    self.df.at[index, 'exits'] = True
                    self.df.at[index, 'price_action'] = row['low']
                    
                    self.order['result'] = 'S/L'
                    self.order['close time'] = row['timestamp']
                    self.order['close'] = self.order['S/L']
                    
                    self.current_action = 'close'
                    self.orders = self.orders.append(self.order, ignore_index = True)
                    if self.plot == True: self.plot_order(self.order)
                    self.reset_order()
                    
                elif row['high'] >= self.order['T/P']:
                    self.df.at[index, 'exits'] = True
                    self.df.at[index, 'price_action'] = row['high']
                    
                    self.order['result'] = 'T/P'
                    self.order['close time'] = row['timestamp']
                    self.order['close'] = self.order['T/P']
                    
                    self.current_action = 'close'
                    self.orders = self.orders.append(self.order, ignore_index = True)
                    if self.plot == True: self.plot_order(self.order)
                    self.reset_order()

            elif self.current_action == 'sell':
                if row['high'] >= self.order['S/L'] and row['low'] <= self.order['T/P']:
                    self.unknow_result.append(row)
                    
                if row['high'] >= self.order['S/L']:
                    self.df.at[index, 'short_exits'] = True
                    self.df.at[index, 'price_action'] = row['high']
                    
                    self.order['result'] = 'S/L'
                    self.order['close time'] = row['timestamp']
                    self.order['close'] = self.order['S/L']
                    
                    self.current_action = 'close'
                    self.orders = self.orders.append(self.order, ignore_index = True)
                    if self.plot == True: self.plot_order(self.order)
                    self.reset_order()
                    
                elif row['low'] <= self.order['T/P']:
                    self.df.at[index, 'short_exits'] = True
                    self.df.at[index, 'price_action'] = row['low']
                    
                    self.order['result'] = 'T/P'
                    self.order['close time'] = row['timestamp']
                    self.order['close'] = self.order['T/P']
                    
                    self.current_action = 'close'
                    self.orders = self.orders.append(self.order, ignore_index = True)
                    if self.plot == True: self.plot_order(self.order)
                    self.reset_order()

        try:
            win_rate = round(len(self.orders[self.orders['result'] == 'T/P']) * 100 / len(self.orders), 2)
            loss_rate = round(len(self.orders[self.orders['result'] == 'S/L']) * 100 / len(self.orders), 2)

            win_orders = int((win_rate * len(self.orders)) / 100 )
            loss_orders = len(self.orders) - win_orders
            gain = (win_orders * self.rr) - loss_orders

            result_info = f"total orders: {len(self.orders)}\nema: {self.ema}, atr: {self.atr}, gain: {gain}\n"
            return {'win_rate': win_rate, 'info': result_info, 'total_order': len(self.orders), 'rr': self.rr, 'gain': gain, 'unknow': self.unknow_result}
        except:
    #         print("No orders to action")
            return {'win_rate': 0, 'info': 'No orders to action'}

In [52]:
pair_test = BackTestATR3X({
    'atr': 14,
    'ema': 260,
    'pip_value': 0.001,
    'rr': 1.5,
    'plot': False,
    'df': df.copy()
})
pair_test.backtest()

{'win_rate': 66.67,
 'info': 'total orders: 60\nema: 260, atr: 14, gain: 40.0\n',
 'total_order': 60,
 'rr': 1.5,
 'gain': 40.0,
 'unknow': [timestamp        2022-01-26 19:00:00+00:00
  open                            114.330002
  high                            114.460999
  low                             114.231003
  close                           114.460999
  adj close                       114.460999
  volumn                                   0
  ATRr_14                           0.071296
  EMA_260                         114.127087
  action                                None
  entries                              False
  exits                                False
  short_entries                        False
  short_exits                          False
  price_action                    114.460999
  Name: 16703, dtype: object,
  timestamp        2022-03-28 09:50:00+01:00
  open                            124.697998
  high                            125.102997
  low                

In [24]:
"""
Grid parameter example
ema: 200 -> 260
atr: 10 -> 30
condition line 1 < line 2 < line 3
"""
def create_grid_params(line1_start, line1_end, line2_start, line2_end):
    params = []
    for i in range(line1_start, line1_end+1):
        for j in range(line2_start, line2_end+1):
            params.append([i, j])
    return params
params_list = create_grid_params(200, 260, 10, 30)
len(params_list)

1281

In [26]:
good_params = {'win': 0, 'info': ''}
start = 0
end = 1281
win_start = 0

loop_num = start
for param in params_list[start:end]:
    obj = BackTestATR3X({'atr': param[1], 'ema': param[0], 'pip_value': 0.001, 'rr': 1.5, 'df': df.copy(), 'plot': False})
    result = obj.backtest()
    if result['win_rate'] > win_start and result['total_order'] > 100:
        win_start = result['win_rate']
        good_params['win'] = result['win_rate']
        good_params['info'] = result['info']
    loop_num = loop_num + 1
    print(f"run: {loop_num} to {end}", end = "\r")
    
print(f"\nwin rate: {good_params['win']} %\n{good_params['info']}")

KeyboardInterrupt: 